# 第7篇：分组统计
分组是我们日常处理和分析数据中经常会使用的操作技巧，比如现在有一个学校某一门科目所有的学生的考试成绩，要以班级为单位进行成绩排序，那么首先需要对所有学生成绩进行班级分组，即分别对每个班的学生成绩进行平均值（求和也可以）计算，最后再按成绩高低对班级进行排序。

## 第一部分：分组过程理解

### 1. 分组过程
分组函数在pandas中是groupby,其实质是split-apply-combine过程，简称SAC。**分组**是指涉及以下一个或多个步骤的过程：
- 拆分数据到基于某些标准组。
- 将功能独立地应用于每个组。
- 将结果合并为数据结构。
其中，拆分步骤是最简单的。实际上，在许多情况下，我们可能希望将数据集分成几组，然后对这些组进行处理。

### 2. 分组应用
在应用步骤中，我们可能希望执行以下操作之一：
- 聚合（Aggregation）：为每个组计算摘要统计量。一些例子：
    - 计算组总和或均值。
    - 计算小组人数/人数。
- 转换（Transformation）：执行一些特定于组的计算并返回索引相似的对象。一些例子：
    - 标准化组内的数据（zscore）。
    - 用从每个组派生的值填充组内的NA。
- 过滤（Filtration）：根据评估为True或False的逐组计算丢弃一些组。一些例子：
    - 丢弃属于只有几个成员的组的数据。
    - 根据组总和或均值过滤数据。

以上各项的某种组合：GroupBy将检查apply步骤的结果，并在不适合上述两种类别的情况下尝试返回明智的组合结果。

## 第二部分：分组函数

导入相关库

In [1]:
import numpy as np
import pandas as pd

数据准备

In [2]:
index = pd.Index(data=["Tom", "Bob", "Mary", "James", "Andy", "Alice", 'Kobe','Yafei'], name="name")
data = {
    "age": [18, 30, 35, 18, np.nan, 30, 37, 25],
    "city": ["北京", "上海", "广州", "深圳", np.nan, " ", "克利夫兰", "晋城"],
    "sex": ["male", "male", "female", "male", "female", "female", "male", "male"],
    "income": [3000, 8000, 8000, 4000, 6000, 7000, 10000, 70000]
}
user_info = pd.DataFrame(data=data, index=index)
user_info

,age,city,sex,income
name,,,,
Tom,18.0,北京,male,3000
Bob,30.0,上海,male,8000
Mary,35.0,广州,female,8000
James,18.0,深圳,male,4000
Andy,NaN,NaN,female,6000
Alice,30.0,,female,7000
Kobe,37.0,克利夫兰,male,10000
Yafei,25.0,晋城,male,70000


### 1. groupby
>  groupby(by=None,axis=0,level=None,as_index: bool = True, sort: bool = True,group_keys: bool = True,squeeze: bool = no_default,observed: bool = False,dropna: bool = True)

#### 根据单列分组：将user_info按性别分组

In [3]:
user_info_sex_group = user_info.groupby(by='sex')
user_info_sex_group

In [4]:
user_info_sex_group.groups

{'female': ['Mary', 'Andy', 'Alice'], 'male': ['Tom', 'Bob', 'James', 'Kobe', 'Yafei']}

In [5]:
user_info_sex_group.get_group('male')

,age,city,sex,income
name,,,,
Tom,18.0,北京,male,3000
Bob,30.0,上海,male,8000
James,18.0,深圳,male,4000
Kobe,37.0,克利夫兰,male,10000
Yafei,25.0,晋城,male,70000


In [6]:
user_info_sex_group.get_group('female')

,age,city,sex,income
name,,,,
Mary,35.0,广州,female,8000
Andy,NaN,NaN,female,6000
Alice,30.0,,female,7000


#### 根据多列分组：将user_info按照性别和年龄分组

In [7]:
user_info_group = user_info.groupby(by=['sex', 'age'])

In [8]:
user_info_group.groups

{('female', 35.0): ['Mary'], ('female', nan): ['Andy'], ('female', 30.0): ['Alice'], ('male', 18.0): ['Tom', 'James'], ('male', 25.0): ['Yafei'], ('male', 30.0): ['Bob'], ('male', 37.0): ['Kobe']}

In [9]:
user_info_group.get_group(("male", 18))

,age,city,sex,income
name,,,,
Tom,18.0,北京,male,3000
James,18.0,深圳,male,4000


#### 关闭排序
默认情况下，groupby 会在操作过程中对数据进行排序。如果为了更好的性能，可以设置 sort=False。

In [10]:
user_info_group = user_info.groupby(by=['sex', 'age'], sort=False)
user_info_group.groups

{('female', 35.0): ['Mary'], ('female', nan): ['Andy'], ('female', 30.0): ['Alice'], ('male', 18.0): ['Tom', 'James'], ('male', 25.0): ['Yafei'], ('male', 30.0): ['Bob'], ('male', 37.0): ['Kobe']}

#### level参数（用于多级索引）和axis参数

In [11]:
user_info.set_index(['sex', 'age']).groupby(level=1, sort=False).groups

{18.0: [('male', 18.0), ('male', 18.0)], 30.0: [('male', 30.0), ('female', 30.0)], 35.0: [('female', 35.0)], 37.0: [('male', 37.0)], 25.0: [('male', 25.0)]}

### 2. 对象属性

#### 查看分组前n行：head
对分组对象使用head函数，返回的是每个组的前几行，而不是数据集前几行

In [12]:
user_info_sex_group.head(2)

,age,city,sex,income
name,,,,
Tom,18.0,北京,male,3000
Bob,30.0,上海,male,8000
Mary,35.0,广州,female,8000
Andy,NaN,NaN,female,6000


#### 查看分组第1行：first
first显示的是以分组为索引的每组的第一个分组信息

In [13]:
user_info_sex_group.first()

,age,city,income
sex,,,
female,35.0,广州,8000
male,18.0,北京,3000


#### 查看指定列
在使用 groupby 进行分组后，可以使用点或切片 [] 操作来完成对某一列的选择

In [14]:
user_info_group.city.head()

name
Tom        北京
Bob        上海
Mary       广州
James      深圳
Andy      NaN
Alice        
Kobe     克利夫兰
Yafei      晋城
Name: city, dtype: object

In [15]:
user_info_group['city'].head()

name
Tom        北京
Bob        上海
Mary       广州
James      深圳
Andy      NaN
Alice        
Kobe     克利夫兰
Yafei      晋城
Name: city, dtype: object

In [16]:
user_info_group[['city', 'income']].head()

,city,income
name,,
Tom,北京,3000
Bob,上海,8000
Mary,广州,8000
James,深圳,4000
Andy,NaN,6000
Alice,,7000
Kobe,克利夫兰,10000
Yafei,晋城,70000


#### 查看指定列的简单统计信息

In [17]:
user_info_sex_group[['age']].mean()

,age
sex,
female,32.5
male,25.6


In [18]:
user_info_sex_group[['age']].sum()

,age
sex,
female,65.0
male,128.0


#### 组个数

In [19]:
user_info_sex_group.ngroups

2

#### 组大小/容量

In [20]:
user_info_sex_group.size()

sex
female    3
male      5
dtype: int64

#### 组索引

In [21]:
user_info_sex_group.groups

{'female': ['Mary', 'Andy', 'Alice'], 'male': ['Tom', 'Bob', 'James', 'Kobe', 'Yafei']}

### 3.  遍历分组

在对数据进行分组后，可以进行遍历。如果是根据多个字段来分组的，每个组的名称是一个元组。

In [22]:
for name, group in user_info_sex_group:
    print(name)
    display(group)

female


,age,city,sex,income
name,,,,
Mary,35.0,广州,female,8000
Andy,NaN,NaN,female,6000
Alice,30.0,,female,7000


male


,age,city,sex,income
name,,,,
Tom,18.0,北京,male,3000
Bob,30.0,上海,male,8000
James,18.0,深圳,male,4000
Kobe,37.0,克利夫兰,male,10000
Yafei,25.0,晋城,male,70000


In [23]:
for name, group in user_info_group:
    print(name)
    display(group)

('male', 18.0)


,age,city,sex,income
name,,,,
Tom,18.0,北京,male,3000
James,18.0,深圳,male,4000


('male', 30.0)


,age,city,sex,income
name,,,,
Bob,30.0,上海,male,8000


('female', 35.0)


...

,age,city,sex,income
name,,,,
Alice,30.0,,female,7000


('male', 37.0)


,age,city,sex,income
name,,,,
Kobe,37.0,克利夫兰,male,10000


('male', 25.0)


,age,city,sex,income
name,,,,
Yafei,25.0,晋城,male,70000


### 4. 选择组元素

In [24]:
user_info_sex_group.get_group("male")

,age,city,income
name,,,
Tom,18.0,北京,3000
Bob,30.0,上海,8000
James,18.0,深圳,4000
Kobe,37.0,克利夫兰,10000
Yafei,25.0,晋城,70000


In [25]:
user_info_group.get_group(("male", 18))

,age,city,sex,income
name,,,,
Tom,18.0,北京,male,3000
James,18.0,深圳,male,4000


### 5. 分组函数的方法

In [26]:
print([attr for attr in dir(user_info_group) if not attr.startswith('_')])

['age', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'city', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'income', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sample', 'sem', 'sex', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'var']


In [27]:
user_info_sex_group.median()

,age,income
sex,,
female,32.5,7000
male,25.0,8000


In [28]:
user_info_sex_group.max()

,age,income
sex,,
female,35.0,8000
male,37.0,70000


由此可见，groupby对象可以使用相当多的函数，灵活程度很高

### 6. 连续性变量分组
**分布分析-cut**
> pd.cut(data['col_names'], bins, labels=None)

In [29]:
bins = [0,20,30,40,100]
labels = ['20岁及以下','21岁到30岁','31岁到40岁','41岁以上']
user_info['年龄分层'] = pd.cut(user_info['age'],bins=bins, labels=labels) #可选label添加自定义标签
user_info.groupby(by=['年龄分层'])['age'].count()

年龄分层
20岁及以下     2
21岁到30岁    3
31岁到40岁    2
41岁以上      0
Name: age, dtype: int64

## 第三部分：聚合、转换和过滤

### 1. 聚合（Aggregation）
分组的目的是为了统计，统计的时候需要聚合，所以我们需要在分完组后来看下如何进行聚合。常见的一些聚合操作有：计数、求和、最大值、最小值、平均值等。想要实现聚合操作，一种方式就是调用 agg 方法。　

#### 常用聚合函数
常用的聚合函数包含：mean/sum/size/count/std/var/sem/describe/first/last/nth/min/max等等，下面看一下示例：

In [30]:
user_info

,age,city,sex,income,年龄分层
name,,,,,
Tom,18.0,北京,male,3000,20岁及以下
Bob,30.0,上海,male,8000,21岁到30岁
Mary,35.0,广州,female,8000,31岁到40岁
James,18.0,深圳,male,4000,20岁及以下
Andy,NaN,NaN,female,6000,NaN
Alice,30.0,,female,7000,21岁到30岁
Kobe,37.0,克利夫兰,male,10000,31岁到40岁
Yafei,25.0,晋城,male,70000,21岁到30岁


In [31]:
user_info_sex_group = user_info.groupby('sex')

获取不同性别下所包含的人数

In [32]:
user_info_sex_group['age'].agg(len)

sex
female    3.0
male      5.0
Name: age, dtype: float64

In [33]:
user_info_sex_group.age.count()

sex
female    2
male      5
Name: age, dtype: int64

In [34]:
user_info_sex_group.age.size()

sex
female    3
male      5
Name: age, dtype: int64

获取不同性别年龄下的各指标数

In [35]:
user_info_group = user_info.groupby(by=['sex', 'age'])

In [36]:
user_info_group.agg(len)

city  income  年龄分层
sex    age                     
female 30.0     1       1     1
       35.0     1       1     1
male   18.0     2       2     2
       25.0     1       1     1
       30.0     1       1     1
       37.0     1       1     1

In [37]:
user_info_group.count()

city  income  年龄分层
sex    age                     
female 30.0     1       1     1
       35.0     1       1     1
male   18.0     2       2     2
       25.0     1       1     1
       30.0     1       1     1
       37.0     1       1     1

In [38]:
user_info_group.size()

sex     age 
female  30.0    1
        35.0    1
male    18.0    2
        25.0    1
        30.0    1
        37.0    1
dtype: int64

获取不同性别下包含的最大的年龄

In [39]:
user_info_sex_group.age.agg(max)

sex
female    35.0
male      37.0
Name: age, dtype: float64

In [40]:
user_info_sex_group.age.agg(np.max)

sex
female    35.0
male      37.0
Name: age, dtype: float64

In [41]:
user_info_sex_group.age.max()

sex
female    35.0
male      37.0
Name: age, dtype: float64

Series 和 DataFrame 都包含了 describe 方法，我们分组后一样可以使用 describe 方法来查看数据的情况。

In [42]:
user_info_sex_group.describe()

age                                                 income           \
       count  mean       std   min    25%   50%    75%   max  count     mean   
sex                                                                            
female   2.0  32.5  3.535534  30.0  31.25  32.5  33.75  35.0    3.0   7000.0   
male     5.0  25.6  8.142481  18.0  18.00  25.0  30.00  37.0    5.0  19000.0   

                                                                
                 std     min     25%     50%      75%      max  
sex                                                             
female   1000.000000  6000.0  6500.0  7000.0   7500.0   8000.0  
male    28653.097564  3000.0  4000.0  8000.0  10000.0  70000.0

In [43]:
user_info_group.describe()

income                                                           \
             count     mean         std      min      25%      50%      75%   
sex    age                                                                    
female 30.0    1.0   7000.0         NaN   7000.0   7000.0   7000.0   7000.0   
       35.0    1.0   8000.0         NaN   8000.0   8000.0   8000.0   8000.0   
male   18.0    2.0   3500.0  707.106781   3000.0   3250.0   3500.0   3750.0   
       25.0    1.0  70000.0         NaN  70000.0  70000.0  70000.0  70000.0   
       30.0    1.0   8000.0         NaN   8000.0   8000.0   8000.0   8000.0   
       37.0    1.0  10000.0         NaN  10000.0  10000.0  10000.0  10000.0   

                      
                 max  
sex    age            
female 30.0   7000.0  
       35.0   8000.0  
male   18.0   4000.0  
       25.0  70000.0  
       30.0   8000.0  
       37.0  10000.0

#### 避免多层索引

如果是根据多个键来进行聚合，默认情况下得到的结果是一个多层索引结构。有两种方式可以避免出现多层索引，先来介绍第一种。对包含多层索引的对象调用 reset_index 方法。

##### 方式一

In [44]:
user_info_group.agg(len)

city  income  年龄分层
sex    age                     
female 30.0     1       1     1
       35.0     1       1     1
male   18.0     2       2     2
       25.0     1       1     1
       30.0     1       1     1
       37.0     1       1     1

In [45]:
user_info_group.agg(len).reset_index()

,sex,age,city,income,年龄分层
0,female,30.0,1,1,1
1,female,35.0,1,1,1
2,male,18.0,2,2,2
3,male,25.0,1,1,1
4,male,30.0,1,1,1
5,male,37.0,1,1,1


##### 方式二
另外一种方式是在分组时，设置参数 as_index=False

In [46]:
user_info.groupby(['sex', 'age'], as_index=False).agg(len)

,sex,age,city,income,年龄分层
0,female,30.0,1,1,1
1,female,35.0,1,1,1
2,male,18.0,2,2,2
3,male,25.0,1,1,1
4,male,30.0,1,1,1
5,male,37.0,1,1,1


#### 应用多个聚合操作
有时候进行分组后，不单单想得到一个统计结果，有可能是多个。比如想统计出不同性别下的一个收入的总和和平均值。

In [47]:
user_info_sex_group["income"].agg([np.sum, np.mean])

,sum,mean
sex,,
female,21000,7000
male,95000,19000


#### 利用元组重命名

In [48]:
user_info_sex_group["income"].agg([('和', np.sum), ('平均值', np.mean)])

,和,平均值
sex,,
female,21000,7000
male,95000,19000


#### 对DataFrame列应用不同的聚合操作
有时候可能需要对不同的列使用不同的聚合操作。例如，想要统计不同性别下人群的年龄的均值以及收入的总和。

In [49]:
user_info_sex_group.agg({"age": np.mean, "income": np.sum}).rename(columns={"age": "age_mean", "income": "income_sum"})

,age_mean,income_sum
sex,,
female,32.5,21000
male,25.6,95000


#### 自定义函数

In [50]:
user_info_sex_group.age.agg(lambda x: x.mean() + 1)

sex
female    33.5
male      26.6
Name: age, dtype: float64

In [51]:
user_info_sex_group.age.agg(lambda x: x.max() - x.min())

sex
female     5.0
male      19.0
Name: age, dtype: float64

#### 利用NamedAgg函数进行多个聚合
注意：不支持lambda函数，但是可以使用外置的def函数

In [52]:
def R1(x):
    return x.max()-x.min()
def R2(x):
    return x.max()-x.median()
user_info_sex_group.agg(min_income1=pd.NamedAgg(column='income', aggfunc=R1),
                           max_income=pd.NamedAgg(column='income', aggfunc='max'),
                           range_income=pd.NamedAgg(column='income', aggfunc=R2)).head()

,min_income1,max_income,range_income
sex,,,
female,2000,8000,1000
male,67000,70000,62000


#### 带参数的聚合函数
判断是否组内年龄至少有一个值在30-40之间：

In [53]:
def f(s,low,high):
    return s.between(low,high).max()
user_info_sex_group['age'].agg(f,30,40)

sex
female    True
male      True
Name: age, dtype: bool

如果需要使用多个函数，并且其中至少有一个带参数，则使用wrap技巧：

In [54]:
def f_test(s,low,high):
    return s.between(low,high).max()
def agg_f(f_mul,name,*args):
    def wrapper(x):
        return f_mul(x,*args)
    wrapper.__name__ = name
    return wrapper
user_info_sex_group['age'].agg([agg_f(f_test,'func_name',30,40),'mean'])

,func_name,mean
sex,,
female,True,32.5
male,True,25.6


### 2. 过滤（Filteration）
filter函数是用来筛选某些组的（务必记住结果是组的全体），因此传入的值应当是布尔标量

In [55]:
user_info_sex_group.filter(lambda x: (x['income'] > 32).all()).head()

,age,city,sex,income,年龄分层
name,,,,,
Tom,18.0,北京,male,3000,20岁及以下
Bob,30.0,上海,male,8000,21岁到30岁
Mary,35.0,广州,female,8000,31岁到40岁
James,18.0,深圳,male,4000,20岁及以下
Andy,NaN,NaN,female,6000,NaN


### 3. 变换（Transformation）
前面进行聚合运算的时候，得到的结果是一个以分组名作为索引的结果对象。虽然可以指定 as_index=False ,但是得到的索引也并不是元数据的索引。如果我们想使用原数组的索引的话，就需要进行 merge 转换。
transform方法简化了这个过程，它会把 func 参数应用到所有分组，然后把结果放置到原数组的索引上（如果结果是一个标量，就进行广播）

#### 组内元素变换
transform函数中传入的对象是组内的列，并且返回值需要与列长完全一致

In [56]:
user_info_sex_group.income.agg(np.mean)

sex
female     7000
male      19000
Name: income, dtype: int64

In [57]:
user_info_sex_group.income.transform(np.mean)

name
Tom      19000
Bob      19000
Mary      7000
James    19000
Andy      7000
Alice     7000
Kobe     19000
Yafei    19000
Name: income, dtype: int64

In [58]:
user_info_sex_group[['age', 'income']].transform(np.mean)

,age,income
name,,
Tom,25.6,19000
Bob,25.6,19000
Mary,32.5,7000
James,25.6,19000
Andy,32.5,7000
Alice,32.5,7000
Kobe,25.6,19000
Yafei,25.6,19000


#### 利用变换方法进行组内标准化

In [59]:
user_info_sex_group['income'].transform(lambda x:(x-x.mean())/x.std()).head()

name
Tom     -0.558404
Bob     -0.383903
Mary     1.000000
James   -0.523504
Andy    -1.000000
Name: income, dtype: float64

#### 利用变换方法进行组内缺失值的均值填充

In [60]:
user_info_sex_group.age.mean()

sex
female    32.5
male      25.6
Name: age, dtype: float64

In [61]:
user_info_sex_group.transform(lambda x: x.fillna(x.mean()))

,age,income
name,,
Tom,18.0,3000
Bob,30.0,8000
Mary,35.0,8000
James,18.0,4000
Andy,32.5,6000
Alice,30.0,7000
Kobe,37.0,10000
Yafei,25.0,70000


## 第四部分：apply函数

除了 上述聚合、过滤和变换操作外，还有更神奇的 apply 操作。  
　　apply 会将待处理的对象拆分成多个片段，然后对各片段调用传入的函数，最后尝试用 pd.concat() 把结果组合起来。func 的返回值可以是 Pandas 对象或标量，并且数组对象的大小不限。  

### transform与apply的比较
**相同点**
- 都能针对dataframe完成特征的计算，并且常常与groupby()方法一起使用。

**不同点**

- apply()里面可以跟自定义的函数，包括简单的求和函数以及复杂的特征间的差值函数等

- transform() 里面不能跟自定义的特征交互函数，因为transform是针对每一元素（即每一列特征操作）进行计算，也就是说在使用 transform() 方法时，需要记得三点：  
    1、它只能对每一列进行计算，所以在groupby()之后，.transform()之前是要指定要操作的列，这点也与apply有很大的不同。

    2、由于是只能对每一列计算，所以方法的通用性相比apply()就局限了很多，例如只能求列的最大/最小/均值/方差/分箱等操作

    3、transform还有什么用呢?最简单的情况是试图将函数的结果分配回原始的dataframe。也就是说返回的shape是（len(df)，1）。注：如果与groupby()方法联合使用，需要对值进行去重

### apply的使用

**apply函数的灵活性**  
> 可能在所有的分组函数中，apply是应用最为广泛的，这得益于它的灵活性，apply函数的灵活性很大程度来源于其返回值的多样性：

#### 标量返回值

In [62]:
user_info_sex_group.apply(np.mean)

,age,income
sex,,
female,32.5,7000.0
male,25.6,19000.0


比如想要统计不同性别最高收入的前n个值，可以通过下面这种方式实现

In [63]:
def f1(ser, num=2):
    return ser.nlargest(num).tolist()
user_info_sex_group["income"].apply(f1)

sex
female      [8000, 7000]
male      [70000, 10000]
Name: income, dtype: object

另外，如果想要获取不同性别下的年龄的均值，通过 apply 可以如下实现。

In [64]:
def f2(sex):
    return sex.age.mean()
user_info_sex_group.apply(f2)

sex
female    32.5
male      25.6
dtype: float64

查看不同性别下收入的最小值

In [65]:
user_info_sex_group['income'].apply(lambda x: x.min())

sex
female    6000
male      3000
Name: income, dtype: int64

#### 列表返回值

查看不同性别下每个人的年龄与最小值之差

In [66]:
user_info_sex_group['income'].apply(lambda x: x - x.min())

name
Tom          0
Bob       5000
Mary      2000
James     1000
Andy         0
Alice     1000
Kobe      7000
Yafei    67000
Name: income, dtype: int64

查看不同性别下每个人的年龄和收入与最小值之差

In [67]:
user_info_sex_group[['income', 'age']].apply(lambda x: x - x.min())

,income,age
name,,
Tom,0.0,0.0
Bob,5000.0,12.0
Mary,2000.0,5.0
James,1000.0,0.0
Andy,0.0,NaN
Alice,1000.0,0.0
Kobe,7000.0,19.0
Yafei,67000.0,7.0


#### DataFrame返回值

In [68]:
user_info_sex_group.apply(lambda x:pd.DataFrame({'age_diff_max':x['age']-x['age'].max(),
                                  'age_diff_min':x['age']-x['age'].min(),
                                  'income_diff_max':x['income']-x['income'].max(),
                                  'income_diff_min':x['income']-x['income'].min()})).head()

,age_diff_max,age_diff_min,income_diff_max,income_diff_min
name,,,,
Tom,-19.0,0.0,-67000,0
Bob,-7.0,12.0,-62000,5000
Mary,0.0,5.0,0,2000
James,-19.0,0.0,-66000,1000
Andy,NaN,NaN,-2000,0


#### 统计多个指标

In [69]:
def f(df):
    data = {}
    data['income_sum'] = df['income'].sum()
    data['income_var'] = df['income'].var()
    data['income_mean'] = df['income'].mean()
    return pd.Series(data)
user_info_sex_group.apply(f)

,income_sum,income_var,income_mean
sex,,,
female,21000.0,1000000.0,7000.0
male,95000.0,821000000.0,19000.0


### agg、transform和apply的效率比较
参考博客：https://www.cnblogs.com/wkang/p/9794678.html
分别计算在同样简单需求下各组合方法的计算时长
- transform() 方法+自定义函数
- transform() 方法+python内置方法
- apply() 方法+自定义函数
- agg() 方法+自定义函数
- agg() 方法+python内置方法

最后得出结论：
> agg+python内置方法 > transform+python内置方法 > agg+自定义函数 >= apply+自定义函数 > transform() 方法+自定义函数

根据此结论，如果我们对分组结果的操作内置函数可以搞定，就用agg或者transform方法，如果内置函数搞不定，需要自定制方法，那么用agg或者apply是最佳选择。

## 第五部分：考试成绩统计
考试成绩如下：
![](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/Python/blog/pandas-index-3.png)
要求：
1. 计算每个学生的总成绩

2. 计算每个学生各学期的总成绩

3. 各门课程平均成绩

4. 各学期大于本课程平均成绩的学生姓名及成绩


整理数据
![](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/Python/blog/pandas-index-4.png)

第一步：读取数据

In [70]:
exam_data = pd.read_excel('data/scores.xlsx')
exam_data.head()

,姓名,课程,学期,成绩
0,王大伟,大学英语,1,92
1,王大伟,大学英语,2,85
2,王大伟,大学英语,3,83
3,王大伟,大学英语,4,90
4,王大伟,高等数学,1,91


### 1. 计算每个学生总成绩

In [71]:
student_total_score = exam_data.groupby(by=['姓名']).agg(
    {'成绩': sum}).rename(columns={'成绩': '总成绩'})
print('1. 学生总成绩：\n', student_total_score)

1. 学生总成绩：
       总成绩
姓名       
孙力   1038
张明   1081
王大伟  1048


### 2. 每个学生各学期的总成绩

In [72]:
student_semester_total = exam_data.groupby(by=['姓名', '学期']).agg(
    {'成绩': sum}).rename({'成绩': ' 总成绩'})
print('\n2. 学生每个学期总成绩：\n', student_semester_total)


2. 学生每个学期总成绩：
          成绩
姓名  学期     
孙力  1   251
    2   255
    3   277
    4   255
张明  1   272
    2   268
    3   276
    4   265
王大伟 1   261
    2   262
    3   261
    4   264


### 3. 各门课程平均成绩

In [73]:
course_avg_score = exam_data.groupby(by=['课程'])['成绩'].mean()
print('\n3. 各门课程平均成绩：\n', course_avg_score)


3. 各门课程平均成绩：
 课程
大学体育    86.333333
大学英语    87.666667
高等数学    89.916667
Name: 成绩, dtype: float64


### 4. 各学期大于本课程平均成绩的学生姓名及成绩

In [74]:
def judge_score(row):
    return row['成绩'] > course_avg_score[row['课程']]
greater_than_avg_student = exam_data[exam_data.apply(judge_score, axis=1)].set_index(keys=['姓名', '课程'])
print('\n4. 各学期大于本课程平均成绩的学生姓名及成绩: \n', greater_than_avg_student)


4. 各学期大于本课程平均成绩的学生姓名及成绩: 
           学期  成绩
姓名  课程          
王大伟 大学英语   1  92
    大学英语   4  90
    高等数学   1  91
    高等数学   3  98
    大学体育   2  91
    大学体育   4  90
孙力  大学英语   3  93
    高等数学   2  93
    大学体育   3  99
    大学体育   4  88
张明  大学英语   1  88
    大学英语   2  94
    大学英语   3  96
    高等数学   1  97
    高等数学   3  94
    大学体育   1  87
    大学体育   4  92


### 5. 将结果输出到文件

In [75]:
# 输出文件
# with pd.ExcelWriter(path="结果.xlsx") as writer:
#     exam_data.to_excel(excel_writer=writer, sheet_name='试题数据', encoding='utf-8', index=False)
#     student_total_score.to_excel(excel_writer=writer, sheet_name='学生总成绩', encoding='utf-8')
#     student_semester_total.to_excel(excel_writer=writer, sheet_name='每个学生各学期总成绩', encoding='utf-8')
#     course_avg_score.to_excel(excel_writer=writer, sheet_name='各门课程平均成绩', encoding='utf-8')
#     greater_than_avg_student.to_excel(excel_writer=writer, sheet_name='各学期大于本课程平均成绩的学生姓名及成绩', encoding='utf-8')
#     writer.save()